## Solving OpenAI Gym Environment - (Taxi-v2)

##### Using Q-learning

Objective is to pick up the passenger from one position and drop them off at another in minimum possible time. 

There are 4 locations (R, G, Y, B) marked in the image. And the task is to pick up the passenger from one of the four locations and drop him off at other. There is a reward of +20 for a successful dropoff, and -1 for every timestep it takes and -10 for illegal pick-up and drop-off actions.

In [2]:
# command to install gym
# !pip install gym

In [30]:
import numpy as np
import random
import time
import gym

In [31]:
# call the environment and store it in 'env' variable
env = gym.make("Taxi-v2") # Create environment

state = env.reset()
env.render()  # helps in visualizing the environment

print("current state is :" ,state)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

current state is : 126


#### Current state
    - yellow: taxi is unoccupied
    - green: taxi is occupied by a passenger
    - blue: passenger
    - magenta: destination
    - other grids: locations

### State Space

The state vector for this problem is (col_index, row_index, destination_locations, passenger_position)
There are 5 rows, 5 columns, 4 destination locations and 5 passenger positions. Therefore, state space = 5x5x4x5 = 500

In [32]:
# Number of possible states
state_size = env.observation_space.n 
print("State space : ", state_size)
#print("Current state : " ,env.env.s)

State space :  500


### Action Space

In [33]:
action_size = env.action_space.n

### Q-learning

In [83]:
Q_table = np.zeros((state_size, action_size))

In [84]:
print(Q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [85]:
# Defining hyperparameters and episodes

learning_rate = 0.1
gamma = 0.8
epsilon = 0.4

episodes = 100000

In [86]:
# Keeping the policy epsilon-greedy
def epsilon_greedy(state, table):
    z = np.random.random() # Randomizes a number to select whether or not to expolit
    
    if z > epsilon:
        action = np.argmax(table[state])    #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        action = env.action_space.sample()    #Exploration: randomly choosing and action
    
    return action

In [87]:
start = time.time()    # tracking time
deltas = []
for episode in range(1,episodes+1):
    
    state = env.reset() # Reset the environment
    done = False        # 'done' defines successfully dropping the passenger off; 
                        # resulting in an end of episode
    Q_change = 0  # to keep a track of difference in the Q-values
        
    while not done:

        action = epsilon_greedy(state, Q_table)
        new_state, reward, done, info = env.step(action)
        
        oldQ_table = Q_table[state, action]
                
        # UPDATE RULE
        Q_table[state, action] += learning_rate * (reward + gamma * 
                                                   np.max(Q_table[new_state,:])-Q_table[state,action])
        
        Q_change = max(Q_change, np.abs(Q_table[state][action] - oldQ_table))
        
        state = new_state
                             
    deltas.append(Q_change)
    
    if episode % 10000 == 0:
        print("Episode: {}".format(episode))
        print(deltas[-10:])
    
#     if deltas[-1] < 0.00000001:
#         break
        
    episode += 1

    
end = time.time()
total_time = end - start
print("Time taken in seconds: ", time)
print("maximum difference: ", deltas[-1])

Episode: 10000
[0.0007653377954426333, 0.05122508145920435, 1.5772921246964833e-05, 1.0805343180209093e-06, 0.0047699159593062035, 0.050756443987424404, 0.06436286067051178, 0.0990585437006537, 0.001050262494089793, 0.025772889071006055]
Episode: 20000
[0.024694136123894594, 0.01044143793009944, 0.0023122941511424244, 0.037949402258600884, 0.0424891128035636, 0.002881838078214294, 0, 0.016594746734374333, 4.6044022816715824e-08, 1.9881417873124363e-08]
Episode: 30000
[4.369837824924616e-13, 5.915268275202834e-13, 0.0004985978867608765, 0.0012500383265461323, 2.382630586161838e-07, 0.05393230513419134, 0.008778018742569671, 1.0892620139202336e-11, 3.155904426677125e-10, 0.058626139558027646]
Episode: 40000
[3.525511793611713e-08, 2.4424906541753444e-15, 2.7364373478278026e-07, 0, 0.06341410601032749, 3.4495338123718966e-06, 2.1562992897017352e-05, 0.0372443128442459, 3.3306690738754696e-16, 0.00028885032726511106]
Episode: 50000
[0, 0.009950616086616293, 5.391459187986669e-07, 0.0001528

In [91]:
print(Q_table)

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -1.6445568   -0.805696    -1.6445568   -0.805696     0.24288
   -9.805696  ]
 [  0.24288      1.5536       0.24288      1.5536       3.192
   -7.4464    ]
 ...
 [  3.19198738   5.24         3.19198981   1.5535861   -5.80799034
   -5.80799949]
 [ -1.64457011  -0.80571269  -1.64459461  -0.805696   -10.64450087
  -10.64411206]
 [ 11.           7.8         11.          15.           2.
    2.        ]]


### Testing Q-learning agent

In [92]:
# Let's change the environment
state = env.reset()  # reset will set the environment to a new and random state
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [93]:
from IPython.display import clear_output
done = False
cumulative_reward = 0

while(done == False):
    
    best_action = np.argmax(Q_table[state,:]) # selecting the best action basis Q-table
    
    # Take the best action and observe the new state and reward
    state, reward, done, info = env.step(best_action) 
    cumulative_reward += reward  
    
    time.sleep(0.5)
    clear_output(wait=True)
    env.render()
    print('Episode Reward = ', cumulative_reward)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Episode Reward =  11
